corss_entropy, one_hot, reshape 메소드 활용

softmax_cross_entropy_with_logits()


Animal Classification
- 동물의 분류표를 이용(data-04-zoo.csv)해서 학습을 시킴. 
- 다리가 몇개인지, 꼬리가 몇개인지 등등 특징을 이용한 분류표.

In [5]:
#데이터를 numpy를 이용해서 읽어옴

# Lab 6 Softmax Classifier
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

'''
(101, 16) (101, 1)
'''
pass

(101, 16) (101, 1)


In [7]:
nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6, shape [n,1]

In [8]:
#one hot 인코딩을 위한 형태 변환
#one_hot 메소드는 rank를 1 높여주는 함수이며 인자를 class(분류, 경우의 수) 개수만큼 
#  늘려서 인코딩된다.
Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot, shape [n,1,7]
print("one_hot:", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # reshape메소드를 이용해 원하는 형태([-1, nb_classes]) 형태로 만들어준다. (호환되는 조건 하에서)
print("reshape one_hot:", Y_one_hot)

#즉, [y, y, y, y, ...]를
# [[xxxxxxx],[xxxxxxx],[xxxxxxx],[xxxxxxx],[xxxxxxx],...]에서(one_hot)
# [xxxxxxx, xxxxxxx, xxxxxxx, xxxxxxx, xxxxxxx, ...]로 만들어준다.(reshape)

one_hot: Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape:0", shape=(?, 7), dtype=float32)


In [9]:
W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

#logit과 Y값(training set)을 인자로 softmax_cross_entropy_with_logits에 전달해준다.
#v2버전 메소드인 점과 stop_gradient메소드를 취하는 부분은 강의내용에선 없던 내용.
#이후 구현 시 확인
# Cross entropy cost/loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                 labels=tf.stop_gradient([Y_one_hot])))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#argmax()에서 axis옵션은 arr의 어떤 축(axis)을 기준으로 최대값을 찾을건지 묻는 옵션이다.
#예를 들어 
# [[1 2 3 4]
#  [2 3 4 5]
#  [4 5 6 7]]에서
#argmax(arr, 0)은 0번째 축, 즉 가장 바깥 벡터에서 따지므로 결과는 [2 2 2 2]이고
#argmax(arr, 1)은 1번째 축, 즉 그 다음 내부 벡터이므로 결과는 [3, 3, 3]이다.
#axis값이 클수록 내부에서 따지고, 작을수록 외부에서 따짐.
#외부에서 따질수록 인자끼리 같은 인덱스마다 따지게 됨. 각 인자의 0번째 인자, 각 인자의 1번째 인자, ...
#https://www.tensorflow.org/api_docs/python/tf/math/argmax

In [10]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        _, cost_val, acc_val = sess.run([optimizer, cost, accuracy], feed_dict={X: x_data, Y: y_data})
                                        
        if step % 100 == 0:
            print("Step: {:5}\tCost: {:.3f}\tAcc: {:.2%}".format(step, cost_val, acc_val))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()): 
        #flatten()은 열벡터를 행벡터로 만들어줌.(즉, 모양적으로 수평적이게 바꾸어줌.)
        #zip은 p, y문에 for문의 인자로 넘겨주기 편하게 하기 위해 사용하는 python 기본 메소드
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

Step:     0	Cost: 5.480	Acc: 37.62%
Step:   100	Cost: 0.806	Acc: 79.21%
Step:   200	Cost: 0.488	Acc: 88.12%
Step:   300	Cost: 0.350	Acc: 90.10%
Step:   400	Cost: 0.272	Acc: 94.06%
Step:   500	Cost: 0.222	Acc: 95.05%
Step:   600	Cost: 0.187	Acc: 97.03%
Step:   700	Cost: 0.161	Acc: 97.03%
Step:   800	Cost: 0.141	Acc: 97.03%
Step:   900	Cost: 0.124	Acc: 97.03%
Step:  1000	Cost: 0.111	Acc: 97.03%
Step:  1100	Cost: 0.101	Acc: 99.01%
Step:  1200	Cost: 0.092	Acc: 100.00%
Step:  1300	Cost: 0.084	Acc: 100.00%
Step:  1400	Cost: 0.078	Acc: 100.00%
Step:  1500	Cost: 0.072	Acc: 100.00%
Step:  1600	Cost: 0.068	Acc: 100.00%
Step:  1700	Cost: 0.064	Acc: 100.00%
Step:  1800	Cost: 0.060	Acc: 100.00%
Step:  1900	Cost: 0.057	Acc: 100.00%
Step:  2000	Cost: 0.054	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: